In [3]:
pip install requests beautifulsoup4 pandas lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

HEADERS = {"User-Agent": "Mozilla/5.0"}

URLS = {
    2023: "https://www.getmyuni.com/exams/gate-exam-marks-vs-rank",
    2024: "https://www.getmyuni.com/exams/gate-exam-marks-vs-rank",
    2025:  "https://www.getmyuni.com/exams/gate-exam-marks-vs-rank"
}

def extract_number(text):
    nums = re.findall(r"\d+", text)
    if len(nums) == 1:
        return int(nums[0])
    elif len(nums) >= 2:
        return (int(nums[0]) + int(nums[1])) // 2  # avg for ranges
    return None

all_data = []

for year, url in URLS.items():
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "lxml")
    tables = soup.find_all("table")

    for table in tables:
        rows = table.find_all("tr")[1:]
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 2:
                marks = extract_number(cols[0].get_text(strip=True))
                rank = extract_number(cols[1].get_text(strip=True))

                if marks is not None and rank is not None:
                    all_data.append({
                        "year": year,
                        "marks": marks,
                        "rank": rank
                    })

df = pd.DataFrame(all_data, columns=["year", "marks", "rank"])

for year in URLS.keys():
    year_df = df[df["year"] == year]
    year_df.to_csv(f"gate_{year}_marks_vs_rank.csv", index=False)

print(df.head())
print("Years scraped:", df["year"].unique())


   year  marks  rank
0  2023     92    80
1  2023     89    77
2  2023     86    73
3  2023     83    70
4  2023     80    65
Years scraped: [2023 2024 2025]


In [75]:
df

,year,marks,rank
0,2023,92,80
1,2023,89,77
2,2023,86,73
3,2023,83,70
4,2023,80,65
...,...,...,...
157,2025,350,64
158,2025,750,58
159,2025,1500,52
160,2025,3500,44


In [77]:
df.to_csv("gate_marks_vs_ranks.csv", index=False)
